# KS-Test

In [1]:
"""
Update Parameters Here
"""
COLLECTION = "Quaks"
P_VAL = 0.001

In [2]:
"""
Created on Mon Sep 13 16:47:06 2021
KS test on table that has minting accounts and rarity data
@author: nbax1
"""

from scipy import stats
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from honestnft_utils import config

"""
Plot params
"""
plt.rcParams.update({"figure.facecolor": "white", "savefig.facecolor": "white"})

"""
Helper Functions
"""


def generateSyntheticDataset(size, maxRarity, mode="default"):
    """
    generates a synthetic dataset for sanity checks
    MODIFY THIS FUNCTION IF YOU'RE ANALYZING A COLLECTION WITH NON-UNIFORM DISTRIBUTION
    inputs:
        size: number of NFTs purchased by synthetic buyer
        maxRarity: should be the number of NFTs in synthetic collection
        mode: default buys at random. If mode is a float then x% of buys will be in top 5% of rarity
    """
    if mode == "default":
        # =modify this to select sequential token_ids or only from a subset of collection
        random_sample = random.sample(range(1, maxRarity), size)

        return np.array(random_sample)
    else:
        num_rigged_buys = int(mode * size)
        rigged_buys = random.sample(range(1, int(maxRarity / 20)), num_rigged_buys)
        random_buys = random.sample(range(1, maxRarity), size - len(rigged_buys))
        return np.array(rigged_buys + random_buys)


def getRarityArray(data, account):
    """
    inputs: dataframe with columns to_account and rarity
    account: the account to get data from
    returns: array with rarity rank of every NFT minted by an account
    """
    return np.array(data[data["to_account"] == account]["rank"])


def cal_average(num):
    """
    inputs:
        data: dataframe with columns to_account and rarity
    """
    sum_num = 0
    for t in num:
        sum_num = sum_num + t

    avg = sum_num / len(num)
    return avg


def find_anomalies(data, threshold=2, num_replicates=1):
    """
    Prints KS test results for every account in collection that was anomalously lucky
    inputs:
        data: dataframe with column 'to_account' for account that minted NFT, and 'rank' for rarity ranking
        threshold: integer for minimum number of NFTs minted by account to be included in analysis
        num_replicates: set to 1 if not generating synthetic datasets (used when rarity is non-uniformly distributed)

    """
    vc = data.to_account.value_counts()

    num_datapoints = len(data)
    grifters_data = []

    for account in vc[vc > threshold].index:
        lowest_list = []

        rarity_array = getRarityArray(data, account)
        num_minted = len(rarity_array)
        num_anomalies = 0
        p_values = []
        for x in range(0, num_replicates):
            """
            #make synthetic dataset to compare to actual data
            synthetic = generateSyntheticDataset(num_minted, num_datapoints)
            """
            # generate uniform distribution
            synthetic = np.array(range(1, num_datapoints + 1))
            ks = stats.kstest(rvs=synthetic, cdf=rarity_array, alternative="less")

            if ks[1] < P_VAL:  # raise and you will get more hits
                num_anomalies += 1
                p_values.append(ks[1])

        if num_anomalies >= num_replicates * 0.8:  # arbitrary threshold
            print(account + "," + str(cal_average(p_values)))
            print(
                "num_transactions: "
                + str(len(data[data["to_account"] == account]["txid"].unique()))
            )
            print("num_minted:" + str(len(data[data["to_account"] == account])))
            # outputs lowest rank for each mint transaction
            for transaction in data[data["to_account"] == account]["txid"].unique():
                lowest_rank = min(data[data["txid"] == transaction]["rank"])
                token_id = data.loc[data["rank"] == lowest_rank, "TOKEN_ID"].values[0]
                lowest_list.append([lowest_rank, token_id])

            print("{rank, token_id}")
            print(lowest_list)
            print("\n")

            # add grifter to dict
            grifter = dict()
            grifter["address"] = account
            grifter["pvalue"] = cal_average(p_values)
            grifter["num_transactions"] = len(
                data[data["to_account"] == account]["txid"].unique()
            )
            grifter["num_minted"] = len(data[data["to_account"] == account])
            grifter["token_list"] = lowest_list
            grifters_data.append(grifter)

    pd.DataFrame.from_records(grifters_data).to_csv(
        f"{config.GRIFTERS_DATA_FOLDER}/{COLLECTION}_grifters.csv", index=False
    )

    return

In [3]:
"""
Generate Report
"""
PATH = f"{config.MINTING_FOLDER}/{COLLECTION}_minting.csv"

data_to_analyze = pd.read_csv(PATH)
print("Number of buyers:" + str(len(data_to_analyze["to_account"].unique())))
print("Lucky Buyer,p")
print("\n")
find_anomalies(data_to_analyze)

Number of buyers:820
Lucky Buyer,p


0xd9d1c2623fbb4377d9bf29075e610a9b8b4805b4,3.3893849843803126e-23
num_transactions: 10
num_minted:181
{rank, token_id}
[[759, 4936], [56, 4979], [226, 5010], [19, 5037], [217, 5045], [150, 5102], [198, 5160], [194, 5478], [105, 5830], [51, 5859]]


0x0008d343091ef8bd3efa730f6aae5a26a285c7a2,4.408372814708358e-17
num_transactions: 8
num_minted:160
{rank, token_id}
[[146, 3497], [452, 3642], [140, 4846], [57, 4850], [397, 4875], [37, 4901], [260, 5636], [161, 5682]]


0xb9b6856efd128294a912d584366448bc3d4ea979,4.554612995930961e-05
num_transactions: 5
num_minted:100
{rank, token_id}
[[958, 4004], [369, 4013], [18, 4033], [340, 4061], [99, 4074]]


0xcd55ac0917fc01fb05600524c7980567d3aff8d5,3.904632988295227e-08
num_transactions: 4
num_minted:70
{rank, token_id}
[[309, 5212], [210, 5342], [53, 5381], [839, 5551]]


0x955807f8cb79c3a0f4f1e20a8eb336c51d37e5ca,4.282430738485098e-12
num_transactions: 2
num_minted:40
{rank, token_id}
[[131, 5776], [100, 592